In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

from aosMetric import opd2psf
from aosMetric import calc_pssn
from aosMetric import psf2delta
from aosMetric import psf2FWHMrms
from aosMetric import psf2FWHMring
from lsst.cwfs.tools import extractArray

In [ ]:
iField = 1 #field center
instring = 'polishing_M1M3SOML_SPOTS'
wfmFile = '../data/%s_fld%d_wfm.fits' % (instring, iField) #wavefront OPD in wave, 2048x2048
psfFile = wfmFile.replace('_wfm','_psf')
IHDU = fits.open(wfmFile)
wfm = IHDU[0].data
IHDU.close()

In [ ]:
plt.imshow(wfm,vmin=-1, vmax=1)
plt.colorbar()
plt.title('OPD in wave')

In [ ]:
# 1mas pixel on the PSF, so that we can look at fine feature, 
# but can still cover large radius with a acceptably large array
pixum = 0.05
pixmas = pixum*20 
d = 4096
wlum = 0.5
wfmum = wfm*wlum
if not os.path.isfile(psfFile):
    img = opd2psf(wfmum, 0, wlum, pixum, 1, 1.2335, 0) #input OPD needs to be in micron
    psf = extractArray(img, d)
    hdu = fits.PrimaryHDU(psf)
    hdu.writeto(psfFile)
        
else:
    IHDU = fits.open(psfFile)
    psf = IHDU[0].data
    IHDU.close()

In [ ]:
plt.imshow(psf,vmax=0.0001)
plt.colorbar()
plt.title('PSF (normalized total intensity)')

In [ ]:
dp1=100
psfp1=(extractArray(psf,dp1))
x0=np.arange(1,dp1+1)
x1=(x0-dp1/2)*pixmas
x, y = np.meshgrid(x1, x1)
psfp1=psfp1/np.max(psfp1)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(x,y,psfp1,cmap=plt.cm.coolwarm)
ax.set_xlabel('mas')
ax.set_ylabel('mas')

In [ ]:
#pssn, and the pssn-native fwhmeff
pssn, fwhmeff_12mas = calc_pssn(wfmum, wlum, type='opd')

In [ ]:
#fwhmeff from pssn. This has been the default method
fwhmeff = np.sqrt((1-pssn)/pssn)*600*1.086

In [ ]:
#fwhmeff of the instrument psf itself.
psfn = psf/np.sum(psf)
neff = 1/np.sum(psfn**2)
fwhmeff_1mas = 0.664*pixmas*np.sqrt(neff)

In [ ]:
# get fwhmdif using d80
fwhm, xbar, ybar = psf2delta(psf,pixum,0,0,'fwhm',0)
fwhm80 = fwhm*20  #from micron to mas

In [ ]:
# get fwhmdif using d99
fwhm, xbar, ybar = psf2delta(psf,pixum,0,0,'fwhm99',0)
fwhm99 = fwhm*20 #from micron to mas

In [ ]:
# get fwhmgeo using rms spot size
fwhm, xbar, ybar, maskR = psf2FWHMrms(psf,-1,0)
fwhmgeo = fwhm*pixmas

In [ ]:
FWHMRing = psf2FWHMring(wfmum, wlum)

In [ ]:
print('PSSN \t \t -> FWHMeff = %6.1f'% fwhmeff)
print('psfe in PSSN \t -> FWHMeff= %6.1f (with 12mas resolution)'% fwhmeff_12mas)
print('psf \t \t -> FWHMeff = %6.1f (with 1mas resolution)'% fwhmeff_1mas)
print('D80 \t \t -> FWHMdif = %6.1f'% fwhm80)
print('D99 \t \t -> FWHMdif = %6.1f'% fwhm99)
print('RMS \t \t -> FWHMgeo = %6.1f'% fwhmgeo)
print('Ring \t \t -> FWHM = %6.1f'% FWHMRing)

In [ ]:
dp2=d
psfp2=(extractArray(psf,dp2))
psfp2 = psfp2/np.max(psfp2)
x0=np.arange(1,dp2+1)
x2=(x0-dp2/2)*pixmas/1000 #in arcsec
plt.semilogy(x2,psfp2[dp2/2,:],'-b');
y80 = np.exp(-x2**2/2/((fwhm80/1000/2.3548)**2))
plt.semilogy(x2,y80,'-r',label='FWHM80')
y99 = np.exp(-x2**2/2/((fwhm99/1000/2.3548)**2))
plt.semilogy(x2,y99,'-k',label='FWHM99')
yeff = np.exp(-x2**2/2/((fwhmeff/1000/2.3548)**2))
plt.semilogy(x2,yeff,'-g',label='FWHMeff')
yring = np.exp(-x2**2/2/((FWHMRing/1000/2.3548)**2))
plt.semilogy(x2,yring,'-m',label='FWHMRing')
plt.ylim([1e-10,1])
plt.xlabel('arcsec')
plt.legend(loc='upper right')